# Exploring Pretraining Datasets

## C4

C4 is a massive dataset used in pre-training T5. In our pre-training setting we only use it as a regularizer so as the model does not forget to understand natural language.

Download link: https://huggingface.co/datasets/allenai/c4/tree/main

In [1]:
import pandas as pd
import json
import random
import gzip
import glob
import numpy as np

from tqdm.notebook import tqdm
from sentence_splitter import split_text_into_sentences
from nltk.tokenize import word_tokenize

### Reading

In [2]:
with open('../storage/datasets/c4/c4_data_original.json') as f:
    c4_original = [json.loads(line)['text'] for line in f]

### Processing

* Removing newlines
* Sentence splitting
* Shuffling

In [3]:
# Remove newlines
c4 = [text.replace('\n', ' ') for text in c4_original]

# Sentence split
c4_sentences = []
for text in tqdm(c4):
    sents = split_text_into_sentences(text, language='en')
    c4_sentences.extend(sents)

random.shuffle(c4_sentences)

del c4

  0%|          | 0/356317 [00:00<?, ?it/s]

## WDC


WebTableCorpus is a collection of tables crawled from the web.

Download link: http://webdatacommons.org/webtables/2015/downloadInstructions.html

## Reading

In [2]:
table_paths = glob.glob('../storage/datasets/wdc/original/1438042981460.12/warc/*')
original_tables = []

def get_tables():
    for table_path in table_paths[:500]:
        with gzip.open(table_path, 'r') as f:
            for line in f:
                try:
                    yield json.loads(line)
                except UnicodeDecodeError:
                    continue
# print(len(original_tables))

## Filtering

In [3]:
def is_english(table):
    return any(domain in table['url'] for domain in ['.com', '.eu', '.uk', '.net', '.org'])

def has_header(table):
    return table['hasHeader']

def is_not_empty(table):
    return len(table['relation']) >= 2

def is_not_huge(table):
    return len(table['relation']) < 50 and len(table['relation'][0]) < 15

def has_title_or_page_title(table):
    return table['title'] != '' or table['pageTitle'] != ''

def title_is_not_huge(table):
    if table['title'] != '' and len(table['title'].split()) < 5:
        return True
    elif table['pageTitle'] != '' and len(table['pageTitle'].split()) < 5:
        return True
    else:
        return False
    

filtered_tables = [table for table in get_tables() 
                  if has_header(table) and 
                  is_not_empty(table) and 
                  is_english(table) and
                  is_not_huge(table) and
                  has_title_or_page_title(table) and
                  title_is_not_huge(table)]

print(len(filtered_tables))

122124


## Analysis of TextBefore and TextAfter

In [26]:
def calculate_token_overlap(text, row):
    text_tokens = word_tokenize(text)
    
    # We want the text to be at least 8 tokens so as it resembles real text
    if len(row) == 0 or len(text_tokens) < 8:
        return 0
    
    token_set = set(text_tokens)
    row_set = set(row)
    
    return len(token_set.intersection(row_set)) / min(len(text_tokens), len(row_set))


def calculate_row_overlaps(table, text_position="textBeforeTable"):
    return [calculate_token_overlap(table[text_position], row) 
            for row in table['relation'][1:]]  # Skip the header


def calculate_dataset_overlaps(tables):
    before_table_max = []
    after_table_max = []
    
    for table in tqdm(tables):
        before_table_max.append(max(calculate_row_overlaps(table, text_position="textBeforeTable")))
        after_table_max.append(max(calculate_row_overlaps(table, text_position="textAfterTable")))

    return before_table_max, after_table_max


with open('../storage/datasets/wdc/filtered/1438042981525.10.json', 'r') as inp:
    filtered_tables = json.load(inp)

before_overlaps, after_overlaps = calculate_dataset_overlaps(filtered_tables)

  0%|          | 0/37056 [00:00<?, ?it/s]

In [27]:
before_overlaps = np.array(before_overlaps)
after_overlaps = np.array(after_overlaps)

## How many tables have at least one row with more than 50% overlap
thresh = 0.4
print(f"More than {thresh} | TextBefore: {np.sum(before_overlaps > thresh) / len(before_overlaps)}")
print(f"More than {thresh} | TextAfter: {np.sum(after_overlaps > thresh) / len(after_overlaps)}")

## What is the average overlap of the text before and the text after?
print(f"Average overlap | TextBefore: {np.mean(before_overlaps)}")
print(f"Average overlap | TextAfter: {np.mean(after_overlaps)}")

More than 0.4 | TextBefore: 0.15104166666666666
More than 0.4 | TextAfter: 0.40282275474956825
Average overlap | TextBefore: 0.14498885856155933
Average overlap | TextAfter: 0.3959055534994654


In [30]:
before_overlaps = np.array(before_overlaps)
after_overlaps = np.array(after_overlaps)

thresh = 0.4
after_inds = np.where(after_overlaps > thresh)[0]
before_inds = np.where(before_overlaps > thresh)[0]


for i, ind in enumerate(before_inds):
    table = pd.DataFrame(filtered_tables[ind]['relation'][1:], columns=filtered_tables[ind]['relation'][0])
    print(filtered_tables[ind]['textBeforeTable'])
    display(table)
    
    print("-" * 90)
    
    if i > 20:
        break
    

Artist's attributes Catálogos Colecciónes Instituciones Representantes Exposiciones Colectivas Exposiciones Individuales Datos Biográficos Análisis Perfil Fila (2015): Simeon Ignatov , Already a member? Login now Buy a membership You will only see limited information for the artist until you purchase a membership. Full Access to this page is restricted to members of ArtFacts.Net. Artistas Exposiciones Instituciones Noticias Instituciones Exposiciones Artistas Contacto Productos y Precios ¿Quiénes somos? Identificación Saber más Acceso gratuito Italiano Français Español English Deutsch ArtFacts.net Please enable JavaScript to view this page correctly


,Artistas,Manoela Ignatova,Eleonora Hadzhinikolova,Velizar Dimchev,Plamen Assenov,Alexander Tsanev
0,Fila,,,,,
1,№,1,1,1,1,1


------------------------------------------------------------------------------------------
Saraperos de Saltillo HR: MXO: Heras 2 (10), Sandoval, Jo 1 (1) W: Valdez, Ro (W, 2-0, 6.16) ; L: Guerrero, D (L, 4-2, 5.30) 0 13 10 x 0 2 0 2 1 0 1 4 Mexico 0 11 3 0 0 1 0 1 0 0 1 0 Saltillo E H R 9 8 7 6 5 4 3 2 1 Final Mexican League (AAA) --Major League Baseball-- American League (MAJ) National League (MAJ) --Triple-A-- International League (AAA) Mexican League (AAA) Pacific Coast League (AAA) --Double-A-- Eastern League (AA) Southern League (AA) Texas League (AA) --High Class A-- California


,Player,"Valdez, Ro (W, 2-0)","Reyes, D","Castaneda, F","Sandoval, Ju"
0,IP,5.0,1.0,2.0,1.0
1,H,6,3,1,1
2,R,2,1,0,0
3,ER,2,1,0,0
4,BB,3,0,0,0
5,SO,2,2,3,0
6,HR,0,0,0,0
7,ERA,6.16,4.91,6.00,4.95


------------------------------------------------------------------------------------------
Midland RockHounds HR: MRO: Carter 1 (), Everidge 1 () W: Deduno (W, 6-1, 2.92) ; L: Cramer (L, 2-3, 4.74) 0 12 8 x 1 0 0 0 0 0 0 7 Tulsa 4 9 3 0 1 0 0 0 0 2 0 0 Midland E H R 9 8 7 6 5 4 3 2 1 Final Texas League (AA) --Major League Baseball-- American League (MAJ) National League (MAJ) --Triple-A-- International League (AAA) Mexican League (AAA) Pacific Coast League (AAA) --Double-A-- Eastern League (AA) Southern League (AA) Texas League (AA) --High Class A-- California League (HiA)


...

------------------------------------------------------------------------------------------
Join us for a 26-mile loop ride around the City of Columbia. Plan to ride a mix of both soft surface trails, bike lanes and streets with low to medium traffic volume. Participants must have intermediate on-road riding skills. Staff will provide SAG (support and gear) for minor maintenance issues. Please make sure you are self-supported with an inner tube (correct size and valve type) or patch kit and water. Helmets required. Optional: Lunch (on your own) after the ride. Registration suggested. Drop-in participants are welcome. 118107 - Loop the City Ride Showing: 1 to 24 Total Results: 24 Activity Search Results Enroll Now Enroll Now All Instructors All Instructors Bishop Steven Walker Ray Availability Option: All Classes Only Available Classes Available Nonresident Classes Time Blocks: Grade: Unspecified Pre-K (0-3) Pre-K (3-5) Kinder 1st 2nd 3rd 4th 5th 6th 7th 8th 9th 10th


,Cart Icon,Read Notice,Read Notice,Read Notice
0,Activity,118302-01,118302-02,118302-03
1,Description,Bicycle Maintenance at Home,Bicycle Maintenance at Home,Bicycle Maintenance
2,Dates,08/20/15 - 08/20/15,09/10/15 - 09/10/15,10/28/15 - 10/28/15
3,Time,6:00P - 7:30P,6:00P - 7:30P,6:00P - 7:30P
4,Days,Th,Th,W
5,Location,Armory Sports Center,Armory Sports Center,Armory Sports Center
6,Fees,$0,$0,$0
7,Ages,14 years and Up,14 years and Up,14 years and Up
8,Status,Unavailable,Unavailable,Unavailable
9,Single Icon,Item Details,Item Details,Item Details


------------------------------------------------------------------------------------------
Join us for a 26-mile loop ride around the City of Columbia. Plan to ride a mix of both soft surface trails, bike lanes and streets with low to medium traffic volume. Participants must have intermediate on-road riding skills. Staff will provide SAG (support and gear) for minor maintenance issues. Please make sure you are self-supported with an inner tube (correct size and valve type) or patch kit and water. Helmets required. Optional: Lunch (on your own) after the ride. Registration suggested. Drop-in participants are welcome. 118107 - Loop the City Ride Showing: 1 to 24 Total Results: 24 Activity Search Results Enroll Now Enroll Now All Instructors All Instructors Bishop Steven Walker Ray Availability Option: All Classes Only Available Classes Available Nonresident Classes Time Blocks: Grade: Unspecified Pre-K (0-3) Pre-K (3-5) Kinder 1st 2nd 3rd 4th 5th 6th 7th 8th 9th 10th


,Cart Icon,Read Notice,Read Notice,Read Notice
0,Activity,118306-01,118306-02,118306-03
1,Description,Fix-a-Flat Class,Fix-a-Flat Class,Fix-a-Flat Class
2,Dates,08/27/15 - 08/27/15,10/13/15 - 10/13/15,11/04/15 - 11/04/15
3,Time,6:00P - 7:30P,6:00P - 7:30P,6:00P - 7:30P
4,Days,Th,Tu,W
5,Location,Armory Sports Center,Armory Sports Center,Armory Sports Center
6,Fees,$0,$0,$0
7,Ages,3 years and Up,3 years and Up,3 years and Up
8,Status,Unavailable,Unavailable,Unavailable
9,Single Icon,Item Details,Item Details,Item Details


------------------------------------------------------------------------------------------


### Check specific cases

In [31]:
# How many tables contain the word Player in first column
player_tables = 0

for table in filtered_tables:
    if table['relation'][0][0] == 'Cart Icon':
        player_tables += 1
        
player_tables

1238

## Analysis of Title and PageTitle

In [74]:
has_title = 0
has_page_title = 0
has_both = 0
has_none = 0

for table in filtered_tables:
    if table['title'] != '':
        has_title += 1
    if table['pageTitle'] != '':
        has_page_title += 1
    if table['title'] != '' and table['pageTitle'] != '':
        has_both += 1
    if table['title'] == '' and table['pageTitle'] == '':
        has_none += 1
        
print(f"Has title: {has_title / len(filtered_tables)}")
print(f"Has page title: {has_page_title / len(filtered_tables)}")
print(f"Has both: {has_both / len(filtered_tables)}")
print(f"Has none: {has_none / len(filtered_tables)}")

Has title: 0.22697422292096558
Has page title: 0.9997707248370509
Has both: 0.22674494775801643
Has none: 0.0


In [75]:
# Explore the difference between title and pageTitle
counter = 0
for table in filtered_tables:
    if table['title'] != '':
        print(table)
        counter += 1
        print("-" * 120)
    if counter > 5:
        break
        
print("!" * 240)
        
counter = 0
for table in filtered_tables:
    if table['pageTitle'] != '':
        print(table)
        counter += 1
        print("-" * 120)
    if counter > 5:
        break

{'relation': [['SEASON', '2008', '2009', 'TOTAL'], ['GP', '16', '6', '23'], ['G', '0', '0', '0'], ['A', '0', '0', '0'], ['PTS', '0', '0', '0'], ['SHOTS', '1', '0', '1'], ['SHOT %', '.000', '.000', '.000'], ['SOG', '1', '0', '1'], ['SOG%', '1.000', '.000', '1.000'], ['GW', '0', '0', '0'], ['PK-ATT', '0-0', '0-0', '0-0']], 'pageTitle': 'Brown', 'title': 'Career Statistics', 'url': 'http://brownbears.com/sports/m-soccer/2010-11/bios/smith_ian00.html', 'hasHeader': True, 'headerPosition': 'FIRST_ROW', 'tableType': 'RELATION', 'tableNum': 1, 's3Link': 'common-crawl/crawl-data/CC-MAIN-2015-32/segments/1438042981460.12/warc/CC-MAIN-20150728002301-00160-ip-10-236-191-2.ec2.internal.warc.gz', 'recordEndOffset': 37217939, 'recordOffset': 37209737, 'tableOrientation': 'HORIZONTAL', 'TableContextTimeStampAfterTable': '{26037=Brown University Athletics | 235 Hope St. Box 1932 | Providence, RI 02912, 21585=Before Brown: Transferred from Western Kentucky University where he started as a freshman ... 